# Pandas Module

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

## Reading a PSIM file

In [ ]:
df1 = pd.read_csv('PD_12_04_2019.csv')

In [ ]:
df1.head()

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html
df1 = df1.set_index('Time')

In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
Vab = df1['Vab']
Vbc = df1['Vbc']

In [ ]:
Vab.head()

In [ ]:
plt.subplot(211)
plt.plot(Vab.index, Vab)
plt.grid(True)
plt.subplot(212)
plt.plot(Vbc.index, Vbc)
plt.grid(True)
plt.show()

In [ ]:
time = np.array(Vab.index)
Vab_a = np.array(Vab)

f, Y = myFFT(time, Vab_a)

In [ ]:
plt.plot(f, Y)
plt.xlim(-1000, 200000)

In [ ]:
plt.plot(f, Y)
plt.xlim(-1000, 200000)
plt.ylim(0, 120)

In [ ]:
plt.plot(f, Y)
plt.xlim(-10, 100)

In [ ]:
time = np.array(Vbc.index)
Vbc_a = np.array(Vbc)

f, Y = myFFT(time, Vbc_a)
plt.plot(f, Y)
plt.xlim(-10, 100)

## TRD and THD das medidas do DataFrame

### Cálculo da THD

$V = 2 \times V_{ab} + \times V_{bc} $

$Y = myFFT(V)$

### $THD \ [\%] = \frac{\sqrt{\sum ^{50}_{h = 2} Y_h}}{Y_1} \times 100$

Onde,

$Y_1$ = Amplitude da fundamental de $2 \times V_{ab} + V_{bc}$

$Y_h$ = Sub-harmônicas de $2 \times V_{ab} + V_{bc}$

In [ ]:
signal = 2 * Vab_a + Vbc_a

print("THD(2 Vab + Vbc) = %.6f %%" % calculateTHD(signal, 6))
# ret = calculateTHD(signal, 6); ret

### Cálculo da TRD

$I = I(Ra)$

$I_{rms} = rmsValue(I)$

$Y = myFFT(I)$

$I_{rated} = \frac{P_{nom}}{3 Vnom}$

### $TRD \ [\%] = \frac{\sqrt{I_{rms}^2 - Y_1^2}}{I_{rated}} \times 100$

Onde,

$I_{rms}$ = Valor RMS de $I(Ra)$

$Y_1$ = Amplitude da fundamental de $I$

$I_{rated}$ = A corrente (RMS) média de cada fase do inversor


In [ ]:
Ira = df1['I(Ra)']

Pnom = 100e3 # VA
Vnom = 220 # V

Irated = Pnom/3/Vnom

print("Considerando Pnom = %d kVA e Vnom = %d V:" % (Pnom/1e3, Vnom))
print("\tTHD(Ira) = %.6f %%" % calculateTRD(Ira, Irated, 6))

In [ ]:
plt.plot(df1.index, Ira)
plt.show()

# Typhoon Module

## Estrutura do código para execução de testes em esquemáticos da Typhoon:

1. Importar a biblioteca  ``typhoon.api.hil``
2. Carregar o modelo **compilado** do esquemático
3. Setar as configurações de **estado inicial do circuito** de comutadores e conversores
4. Setar as configurações **iniciais de simulação** (tempo, canais, passo de captura)
5. Realizar a **captura**
    1. Pode se durante a captura realizar **mudanças no estado** dos comutadores
6. Obter os dados da captura e processá-los 

In [ ]:
# 1. Importar a biblioteca  ``typhoon.api.hil``

import typhoon.api.hil as hil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1° Caso) Boost Closed Loop

In [ ]:
# 2. Carregar o modelo **compilado** do esquemático

filename = "boost_closed_loop"

model_path = '{filename} Target files/{filename}.cpd'.format(filename=filename)

hil.load_model(file=model_path, vhil_device=True)

In [ ]:
hil.get_contactors()

In [ ]:
hil.set_contactor?

In [ ]:
hil.get_pe_switching_blocks()

In [ ]:
hil.set_pe_switching_block_control_mode?

In [ ]:
hil.get_sources()

### Documentação

In [ ]:
# Documentação do Boost Converter

import webbrowser

webbrowser.open("file:///C:/Typhoon%20HIL%20Control%20Center/"+
                "typhoon/help/t-ug003/References/boost_converter.html")

In [ ]:
hil.get_analog_signals()

In [ ]:
hil.get_digital_signals()

In [ ]:
# 3. Setar as configurações de **estado inicial do circuito** de comutadores e conversores

hil.set_contactor('Load', swControl=True, swState=False)
hil.set_pe_switching_block_control_mode(
    blockName='BOOST', switchName='S1', swControl=False
)
hil.set_source_constant_value('Vin', 35)

In [ ]:
# 4. Setar as configurações **iniciais de simulação** (tempo, canais, passo de captura)
# 5. Realizar a **captura**

channels = ['Iout', 'BOOST.PWM_Modulator_BOT_1', 'Vout', 'BOOST.PWM_Modulator_TOP_1', 'Load_fb']
duration = 5
cap_step = 2e-6
rate = int(1//cap_step)

hil.start_simulation()

start_capture(duration, rate, channels)

print('Waiting a sec')
hil.wait_sec(1)
print('Sec waited')

hil.set_contactor('Load', True, True)

df = get_capture_results(wait_capture=True)

hil.stop_simulation()

In [ ]:
df1 = df.copy()
df1.index /= np.timedelta64(1, 's')

In [ ]:
df1['Iout'].plot()

In [ ]:
df1['Vout'].plot()

In [ ]:
df1['Load_fb'].plot()

In [ ]:
hil.set_contactor('Load', swControl=True, swState=False)
hil.set_contactor('Load', True, True, executeAt=1)
hil.set_contactor('Load', True, False, executeAt=2)

In [ ]:
channels = ['Iout', 'BOOST.PWM_Modulator_BOT_1', 'Vout', 'BOOST.PWM_Modulator_TOP_1', 'Load_fb']
duration = 5
cap_step = 2e-6
rate = int(1//cap_step)

hil.start_simulation()

start_capture(duration, rate, channels)

hil.set_contactor('Load', True, False, executeAt=2)

df = get_capture_results(wait_capture=True)

hil.stop_simulation()

In [ ]:
df1 = df.copy()
df1.index /= np.timedelta64(1, 's')

In [ ]:
df1['Iout'].plot()

In [ ]:
df1['Vout'].plot()

In [ ]:
df1['Load_fb'].plot()

## 2° Caso) Two-Level Inverter

In [ ]:
filename = "two_level_carrier_based"

model_path = '{filename} Target files/{filename}.cpd'.format(filename=filename)

hil.load_model(file=model_path, vhil_device=True)

In [ ]:
hil.get_analog_signals() + hil.get_digital_signals() + \
hil.get_pe_switching_blocks() + hil.get_contactors() + \
hil.get_sources() + hil.get_scada_inputs()

### Documentação

In [ ]:
# Documentação do Two Level Rectifier

import webbrowser

webbrowser.open("file:///C:/Typhoon%20HIL%20Control%20Center/" +
                "typhoon/help/t-ug003/References/"+
                "three-phase_two-level_inverter_rectifier.html")

In [ ]:
for switch_name in ['Sa_top', 'Sa_bot', 'Sb_top', 'Sb_bot', 'Sc_top', 'Sb_bot']:
    hil.set_pe_switching_block_control_mode(
        blockName='Three Phase Inverter1', switchName=switch_name, swControl=False
    )

hil.set_source_constant_value('Vs1', 800)

hil.set_scada_input_value('inductance', 50e-6)
hil.set_scada_input_value('resistance', 100)
hil.set_scada_input_value('modulation index', 1.0)

In [ ]:
sa_bot = 'Three Phase Inverter1.IGBT Leg1.PWM_Modulator_BOT_1'
sb_bot = 'Three Phase Inverter1.IGBT Leg2.PWM_Modulator_BOT_1'
sc_bot = 'Three Phase Inverter1.IGBT Leg3.PWM_Modulator_BOT_1'

channels = ['Vab', 'Vbc', 'Ia1', 'Ib1', 'Ic1', #'Three Phase Inverter1.IGBT Leg1.PWM_Modulator_BOT_1', 
            sa_bot, sb_bot, sc_bot]

duration = 5
rate = 200e3 #Hz

hil.start_simulation()

start_capture(duration, rate, channels)

df = get_capture_results(wait_capture=True)

hil.stop_simulation()

In [ ]:
df1 = df.copy()
df1.index /= np.timedelta64(1, 's')

In [ ]:
df1['Vab'].plot()

In [ ]:
# Slicing a dataframe
df1 = df1[:5/60]

In [ ]:
df1.tail()

In [ ]:
df1['Vab'].plot()

In [ ]:
plt.plot(df1.index, 2 * df1['Vab'] + df1['Vbc'])

In [ ]:
print('THD(2 * Vab + Vbc) = %.6f %%' % calculateTHD(2 * df1['Vab'] + df1['Vbc'], 5))

In [ ]:
Pnom = 100e3 # VA
Vnom = 220 # V

I_rated = Pnom/3/220

print("Considerando Pnom = %d kVA e Vnom = %d V:\n" % (Pnom/1e3, Vnom))

TRD_1 = calculateTRD(df1['Ia1'], I_rated, 5)
print('TRD(Ia1) = %.6f %%' % calculateTRD(df1['Ia1'], I_rated, 5))

TRD_2 = calculateTRD(df1['Ib1'], I_rated, 5)
print('TRD(Ib2) = %.6f %%' % calculateTRD(df1['Ib1'], I_rated, 5))

TRD_3 = calculateTRD(df1['Ic1'], I_rated, 5)
print('TRD(Ic1) = %.6f %%' % calculateTRD(df1['Ic1'], I_rated, 5))

print('\nTRD Médio (Ia1, Ib1, Ic1) = %.6f %%' % np.mean([TRD_1, TRD_2, TRD_3]))

In [ ]:
plt.plot(df1.index, df1['Ia1'])
plt.plot(df1.index, df1['Ib1'])
plt.plot(df1.index, df1['Ic1'])
plt.show()

In [ ]:
df1 = df.copy()
df1.index /= np.timedelta64(1, 's')

df1 = df1[5/60: 10/60]

In [ ]:
plt.plot(df1.index, df1['Ia1'])
plt.plot(df1.index, df1['Ib1'])
plt.plot(df1.index, df1['Ic1'])
plt.show()

In [ ]:
print('THD(2 * Vab + Vbc) = %.6f %%' % calculateTHD(2 * df1['Vab'] + df1['Vbc'], 5))

In [ ]:
Pnom = 100e3 # VA
Vnom = 220 # V

I_rated = Pnom/3/220

print("Considerando Pnom = %d kVA e Vnom = %d V:\n" % (Pnom/1e3, Vnom))

TRD_1 = calculateTRD(df1['Ia1'], I_rated, 5)
print('TRD(Ia1) = %.6f %%' % calculateTRD(df1['Ia1'], I_rated, 5))

TRD_2 = calculateTRD(df1['Ib1'], I_rated, 5)
print('TRD(Ib2) = %.6f %%' % calculateTRD(df1['Ib1'], I_rated, 5))

TRD_3 = calculateTRD(df1['Ic1'], I_rated, 5)
print('TRD(Ic1) = %.6f %%' % calculateTRD(df1['Ic1'], I_rated, 5))

print('\nTRD Médio (Ia1, Ib1, Ic1) = %.6f %%' % np.mean([TRD_1, TRD_2, TRD_3]))

In [ ]:
plt.plot(df1.index, df1[sa_bot], label='sa_bot')
plt.plot(df1.index, df1[sb_bot] - 1.25, label='sb_bot')
plt.plot(df1.index, df1[sc_bot] - 2.5, label='sc_bot')

min = df1.index[0]
plt.xlim(min, min+.1/60)
plt.legend()

#### Exercício: Desenvolver um contador de comutações por segundo

In [ ]:
comA = np.array(df1[sa_bot])[:200]
plt.plot(comA)

In [ ]:
aux1 = comA[:-1]
aux2 = comA[1:]

diff = aux2 - aux1
plt.plot(diff)

In [ ]:
diff_abs = np.abs(diff)
plt.plot(diff_abs)

In [ ]:
n_com = np.sum(diff_abs == True); n_com

In [ ]:
def commutationCount(signal, n_cycles, fnom=60):
    aux1 = signal[:-1]
    aux2 = signal[1:]

    diff = aux2 - aux1
    
    print()
    
    diff_abs = np.abs(diff)

    n_com = np.sum(diff_abs == True)
    
    return n_com * fnom/n_cycles

comA = np.array(df1[sa_bot])
commutationCount(comA, 5)

**Espero terminar Aqui!**

## 3° Caso) NPC

In [ ]:
filename = "NPC_SVM"

model_path = '{filename} Target files/{filename}.cpd'.format(filename=filename)

hil.load_model(file=model_path, vhil_device=True)

In [ ]:
hil.get_sources() + hil.get_pe_switching_blocks() + \
hil.get_scada_inputs() + hil.get_contactors() + \
hil.get_analog_signals() + hil.get_digital_signals()

### Documentação

In [ ]:
# Documentação do NPC Leg

import webbrowser

webbrowser.open("file:///C:/Typhoon%20HIL%20Control%20Center/" +
                "typhoon/help/t-ug003/References/npc_leg.html")

In [ ]:
legs = ['LegA', 'LegB', 'LegC']
switchs = ['S1', 'S2', 'S3', 'S4']

for leg in legs:
    for switch in switchs:
        hil.set_pe_switching_block_control_mode(
            blockName=leg, switchName=switch, swControl=False
        )

hil.set_source_constant_value('Vs1', 800)

hil.set_scada_input_value('inductance', 50e-6)
hil.set_scada_input_value('resistance', 100)
hil.set_scada_input_value('modulation index', 1.0)

In [ ]:
sa_bot_1 = 'LegA.PWM_Modulator_BOT_1'
sa_bot_2 = 'LegA.PWM_Modulator_BOT_2'
sb_bot_1 = 'LegB.PWM_Modulator_BOT_1'
sb_bot_2 = 'LegB.PWM_Modulator_BOT_2'
sc_bot_1 = 'LegC.PWM_Modulator_BOT_1'
sc_bot_2 = 'LegC.PWM_Modulator_BOT_2'

channels = ['Vab', 'Vbc', 'Ia1', 'Ib1', 'Ic1',
            sa_bot_1, sa_bot_2, sb_bot_1, sb_bot_2, sc_bot_1, sc_bot_2]

duration = 5
rate = 200e3 #Hz

hil.start_simulation()

start_capture(duration, rate, channels)

df = get_capture_results(wait_capture=True)

hil.stop_simulation()

In [ ]:
df1 = df.copy()
df1.index /= np.timedelta64(1, 's')

df1 = df1[:12/60]

In [ ]:
plt.subplot(211)
plt.plot(df1.index, df1.Vab)
plt.grid(True)
plt.subplot(212)
plt.plot(df1.index, df1.Vbc)
plt.grid(True)
plt.show()

In [ ]:
plt.plot(df1.index, df1.Ia1)
plt.plot(df1.index, df1.Ib1)
plt.plot(df1.index, df1.Ic1)
plt.grid(True)
plt.show()

In [ ]:
TRD_1 = calculateTRD(df1['Ia1'], I_rated, 5)
TRD_2 = calculateTRD(df1['Ib1'], I_rated, 5)
TRD_3 = calculateTRD(df1['Ic1'], I_rated, 5)

print('TRD Médio (Ia1, Ib1, Ic1) = %.6f %%' % np.mean([TRD_1, TRD_2, TRD_3]))

In [ ]:
df1 = df.copy()
df1.index /= np.timedelta64(1, 's')

df1 = df1[4/60:16/60]

In [ ]:
plt.plot(df1.index, df1.Ia1)
plt.plot(df1.index, df1.Ib1)
plt.plot(df1.index, df1.Ic1)
plt.grid(True)
plt.show()

In [ ]:
Pnom = 100e3 # VA
Vnom = 220 # V

I_rated = Pnom/3/220

print("Considerando Pnom = %d kVA e Vnom = %d V:\n" % (Pnom/1e3, Vnom))

TRD_1 = calculateTRD(df1['Ia1'], I_rated, 5)
print('TRD(Ia1) = %.6f %%' % calculateTRD(df1['Ia1'], I_rated, 12))

TRD_2 = calculateTRD(df1['Ib1'], I_rated, 5)
print('TRD(Ib2) = %.6f %%' % calculateTRD(df1['Ib1'], I_rated, 12))

TRD_3 = calculateTRD(df1['Ic1'], I_rated, 5)
print('TRD(Ic1) = %.6f %%' % calculateTRD(df1['Ic1'], I_rated, 12))

print('\nTRD Médio (Ia1, Ib1, Ic1) = %.6f %%' % np.mean([TRD_1, TRD_2, TRD_3]))

In [ ]:
print('THD(2 * Vab + Vbc) = %.6f %%' % calculateTHD(2 * df1['Vab'] + df1['Vbc'], 12))

## 4° Caso)  Droop

In [ ]:
# MOSTRAR NO SPYDER